In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
# contains all code for encoding/processing english to french data from canadian senate
from data_utils import *

In [144]:
# contains all model definition code, training routines, MORE ...
from model_utils import *

# Basic encoder-decoder LSTM

In [46]:
# makes an encoder decoder lstm 
enc_dec_lstm, basic_enc_model, basic_dec_model = basic_lstm(len(english_tokens), len(french_tokens), 256)

In [88]:
enc_dec_lstm.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None, 29)]   0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None, 34)]   0                                            
__________________________________________________________________________________________________
lstm_8 (LSTM)                   [(None, 256), (None, 292864      input_9[0][0]                    
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, None, 256),  297984      input_10[0][0]                   
                                                                 lstm_8[0][1]               

In [ ]:
# fits that model
fit_model(enc_dec_lstm, './models/enc_dec_lstm.h5', *train)

### Deep bidirectional LSTM

In [161]:
# makes the model
deep_lstm, enc_model, dec_model = make_deep_lstm(len(english_tokens), len(french_tokens))

In [162]:
deep_lstm.summary()

Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_89 (InputLayer)           [(None, None, 29)]   0                                            
__________________________________________________________________________________________________
bidirectional_44 (Bidirectional [(None, None, 256),  161792      input_89[0][0]                   
__________________________________________________________________________________________________
input_90 (InputLayer)           [(None, None, 34)]   0                                            
__________________________________________________________________________________________________
bidirectional_45 (Bidirectional [(None, 256), (None, 394240      bidirectional_44[0][0]           
                                                                 bidirectional_44[0][1]    

In [166]:
fit_model(deep_lstm, './models/deep_lstm/deep_lstm_full.h5', *train)

Train on 9467 samples, validate on 2367 samples
Epoch 1/10
9467/9467 [==============================] - 2661s 281ms/sample - loss: 0.1524 - val_loss: 0.1048
Epoch 2/10
9467/9467 [==============================] - 2676s 283ms/sample - loss: 0.0565 - val_loss: 0.0109
Epoch 3/10
9467/9467 [==============================] - 2715s 287ms/sample - loss: 0.0032 - val_loss: 1.4899e-04
Epoch 4/10
9467/9467 [==============================] - 2741s 290ms/sample - loss: 4.5959e-04 - val_loss: 2.1874e-05
Epoch 5/10
9467/9467 [==============================] - 2739s 289ms/sample - loss: 4.8202e-05 - val_loss: 1.0395e-05
Epoch 6/10
9467/9467 [==============================] - 2702s 285ms/sample - loss: 1.6269e-04 - val_loss: 5.2601e-06
Epoch 7/10
9467/9467 [==============================] - 2784s 294ms/sample - loss: 1.8589e-06 - val_loss: 6.3955e-07
Epoch 8/10
 384/9467 [>.............................] - ETA: 43:27 - loss: 5.9757e-07

KeyboardInterrupt: 

In [167]:
deep_lstm.save('./models/deep_lstm/deep_lstm_full.h5')
enc_model.save('./models/deep_lstm/deep_lstm_enc.h5')
dec_model.save('./models/deep_lstm/deep_lstm_dec.h5')

In [169]:
# sample output
s = 180
l = 5

num_decoder_tokens = len(french_tokens)
num_encoder_tokens = len(english_tokens)

def decode_french(pred):
    res = ''
    for p in pred:
        ind = np.argmax(p)
        res += french_tokens[ind]
    return res

# translates a sentence
def ex(ex_num):
    es = train[0][ex_num]
    fs = train[1][ex_num]
    fs_hat = translate(enc_model, dec_model, np.array([es]), len(train_raw[ex_num][0]), french_tokens, True)
    print("English: " + train_raw[ex_num][0])
    print("French (acutal): " + train_raw[ex_num][1])
    pred = deep_lstm.predict([np.array([es]), np.array([fs])])
    print("French (model + feedback): " + decode_french(pred[0]) + "\n")
    print("French (model): " + fs_hat + "\n")

for i in range(s, s+l):
    print("example: " + str(i))
    print("="*20)
    ex(i)
    print("="*20)

example: 180
English: 	honourable senators taking a leaf from senator strattons book i too have a bit of a preamble to my question  

French (acutal): 	honorables sénateurs imitant en cela le sénateur stratton jaimerais moi aussi ajouter un préambule à ma question  

French (model + feedback): honorables sénateurs imitant en cela le sénateur stratton jaimerais moi aussi ajouter un préambule à ma question  
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              